# Trainers

Designing a training loop for learning a complex dataset.

Almondnet currently run for 30 epochs

In [ ]:
import os

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
from torchvision import datasets, transforms

import lib.histopathology as hcd
from lib.dataset import PCam
from lib.evaluate import evaluate
from lib.models import AlmondNet
from lib.train import train
from lib.transforms import ToNormalized, ToClosed


In [ ]:
os.chdir('C:/Users/Amod/PycharmProjects/CS184a/histopathology')
print(os.path.isdir('./models'))
DATASET_PATH = 'C:/Users/Amod/PycharmProjects/CS184a/input'

In [ ]:
num_epochs = 5
num_classes = 2
batch_size = 50
learning_rate = 0.001
rgb_means = [0.7024860505690291, 0.5462877872713974, 0.6964813026563755]
rgb_stds = [0.23890759190102304, 0.28214205984973445, 0.21625778862043862]

In [ ]:
image_dir = os.path.join(DATASET_PATH, 'train')
csv_path = os.path.join(DATASET_PATH, 'train_labels.csv')
pcam_dataset = PCam(image_dir, csv_path, 
                    transforms.Compose([
                        ToClosed(),
                        transforms.RandomVerticalFlip(),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomRotation(93),
                        transforms.CenterCrop(48),
                        transforms.ToTensor(),
                        hcd.transforms.ToNormalized(rgb_means, rgb_stds)
                    ]))
print(len(pcam_dataset))

In [ ]:
train_set, val_set = utils.data.random_split(pcam_dataset, [154000, 66025])

In [ ]:
train_loader = utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=0)
val_loader = utils.data.DataLoader(val_set, batch_size=batch_size, num_workers=0)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

In [ ]:
model = AlmondNet(2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)
print(str(model))

In [ ]:
# uncomment to load from checkpoint

trainer = hcd.training.Trainer(model, train_loader, model_dir='C:/Users/Amod/PycharmProjects/CS184a/histopathology/models')
trainer.load_checkpoint()

In [ ]:
total_loss = train(model, train_loader, device, criterion, optimizer, num_epochs=num_epochs)

In [ ]:
# plt.semilogy()
# plt.plot(tuple(range(num_epochs)), losses, 'b-')
# plt.title('Model Loss')
# plt.xlabel('Epoch')
# plt.ylable('Cross Entropy Loss')
# plt.show()
print(total_loss)

In [ ]:
score, accuracy, loss = hcd.evaluation.evaluate(model, val_loader, device, criterion)
print('Evaluating AlmondNet on Validation set:')
print('F1-Score:', score)
print('Accuracy:', accuracy)
print('Loss:', loss)

# Testing small batches of 100

In [ ]:
since = time.time()
num_steps = len(train_loader)
total_loss = 0
ESL = nd.array[3,100]
for epoch in range(1, num_epochs+1):
    for i, (images, labels) in enumerate(train_loader, start=1):
        images = images.to(self.device)
        labels = labels.to(self.device)
        # Generate prediction and evaluate
        outputs = self.model(images)
        loss = criterion(outputs, labels.long().flatten())
        # Backpropagate loss and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Compute running average of epoch loss
        total_loss += float(loss)
        # Print progress each step
        print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{num_steps}], Loss: {loss.item():.6f}')
        # Print progress every 1000 batches
        
        if i % 100 == 0:
            break    


# Evaluate a saved model

In [ ]:
print(model.__class__.__name__ + '_ckpt.pth')
trainer = hcd.training.Trainer(model, train_loader, model_dir='./models/')
trainer.load_checkpoint()

In [ ]:
score, accuracy, loss = hcd.evaluation.evaluate(model, val_loader, device, criterion)
print('Evaluating AlmondNet on Validation set:')
print('F1-Score:', score)
print('Accuracy:', accuracy)
print('Loss:', loss)

In [ ]:
torch.save(model.state_dict(), './models/AlmondNet_exp1.pth.tar')